#### 导入数据集

In [41]:
import pandas as pd
import os

# 假设Excel文件都在一个名为'data'的文件夹下
folder_path = '../qwen/all_data/excel'
excel_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.xlsx')]

data_frames = []
for file in excel_files:
    df = pd.read_excel(file)
    data_frames.append(df)

combined_data = pd.concat(data_frames, ignore_index=True)

In [70]:
print(combined_data['answer'][6])

  光伏发电具有显著的能源、环保和经济效益，是最优质的绿色能源之一。
  在我国平均日照条件下，安装 1 千瓦分布式光伏系统，1 年可发出 1200 度电，可减少煤炭（标准煤）使用量约 400 千克，减少二氧化碳排放约 1 吨。
  根据世界自然基金会（WWF）研究结果：从减排二氧化碳效果而言，安装 1 平方米光伏发电系统相当于植树造林 100 平方米。目前，发展光伏发电等可再生能源将是根本上解决雾霾、酸雨等环境问题的有效手段之一。


In [71]:
combined_data['question'][6]

'7、为什么说光伏电力是绿色低碳能源？'

#### 加载Baichuan大模型

In [1]:
from langchain_community.llms import BaichuanLLM

baichuan_llm = BaichuanLLM(
    model='Baichuan3-Turbo-128k',
    temperature = 0.01,
    max_tokens = 4096
)

baichuan_llm("你好！")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19928\747844592.py:9: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baichuan_llm("你好！")


'你好！有什么可以帮助你的吗？'

#### 构造扩充样本的prompt

In [2]:
from langchain import PromptTemplate

# template = """请你根据以下 “问题 - 答案” 形式的文本对，生成新的 “问题 - 答案” 文本对。新文本对需保持基本内容不变，只是换一种说法来表达。字数与原文本对保持一致，部分内容可以重复原文本对中的。
# 输出格式要求：
# [
# {{
# "question": "生成的新问题",
# "answer": "生成的新答案"
# }}
# ]
# 示例：
# 原文本对：
# 问题：苹果是什么颜色的？
# 答案：苹果通常是红色、绿色或黄色的。
# 生成的输出示例：
# [
# {{
# "question": "苹果一般呈现出哪些颜色呢？",
# "answer": "苹果常见的颜色有红色、绿色以及黄色。"
# }}
# ]
# 请基于以下原文本对进行生成：
# 问题：{question}
# 答案：{answer}
# """
   
template = """请对以下文本进行改写，改写后的文本需要保持原文的基本意思，同时字数应与原文大致相同。直接输出改写后的文本即可，不要多说废话。

原文本：{context}

改写后的文本："""     
prompt_template = PromptTemplate(
            template=template,
            input_variables = ["context"]
            )


#### 开始生成新样本

In [5]:
data = {
    "question":[],
    "answer":[]
}
new_sample = pd.DataFrame(data)
new_sample

,question,answer


In [3]:
import  pandas as pd
# origin_data = pd.read_excel("../qwen/all_data/excel/电力现货市场百问百答.xlsx")
origin_data = pd.read_excel("../qwen/all_data/excel/电网知识问答.xlsx")
origin_data

,question,answer
0,1.电力系统由什么组成?,组成电力工业的发电及其动力系统、输电、变电、配电、用电设备，也包括调相调压、限制短路电流...
1,2.什么是电网?,一般我们把电力系统中的发电、输电、变电、配电等一次系统及相关继电保护、计量和自动化等二次...
2,3.电网的功能是什么?现代电网具有哪些特点？,电网是电源和用户之间的纽带，其主要功能就是把电能安全、优质、经济地送到用户。电力工业发展...
3,4.如何从宏观上对电网进行分析?,组成电网的发电、输电、变电、配电、用电等设备，从电路原理上理解，无外乎是由电阻、电抗和电...
4,5.什么是电路?如何组成?,电路简单地说就是电流流通的路径，它由若干个电气设备或部件按照一定的方式组合起来，包括电源...
...,...,...
168,169.什么是电力系统的运行操作？,电力系统中的设备有运行、热备用、冷备用和检修四种不同的状态，运行操作是指变更电力系统设备...
169,170.倒闸操作的分类有哪些？有哪些要求？,倒闸操作从满足新技术应用和经济性，即减人增效提高劳动生产率的要求来考虑，倒闸操作可分为就...
170,171.什么是遥控操作和程序操作？有哪些要求？,遥控操作是指从调度端或集控站发出远方操作指令，以微机监控系统或变电站的RTU当地功能为...
171,172.倒闸操作的基本要求是什么？包括哪些内容？,倒闸操作是一项严谨的工作，应根据值班调度员或运行值班负责人的指令，在受令人复诵无误后执行...


In [4]:
import time

data = {
    "question":[],
    "answer":[]
}

for index,(question, answer) in enumerate(zip(origin_data['question'],origin_data['answer'])):
    try:
        question_prompt = prompt_template.format(context=question)
        question_output = baichuan_llm(question_prompt)
        time.sleep(1)
        answer_prompt = prompt_template.format(context=answer)
        answer_output = baichuan_llm(answer_prompt)
        time.sleep(1) 
        # num = len(new_sample)
        # new_sample.loc[num]=[question_output,answer_output]
        data['question'].append(question_output)
        data['answer'].append(answer_output)
        print(f"第{index+15}条新样本已生成......")
        if  index and index%5 == 0:
            time.sleep(30)
    except Exception as e: # 捕获任何异常，打印错误信息并继续下一次循环
        print(f"第{index+129}个新样本生成错误****** \n{e}")
        continue
len(data['question'])

第15条新样本已生成......
第16条新样本已生成......
第17条新样本已生成......
第18条新样本已生成......
第19条新样本已生成......
第20条新样本已生成......
第21条新样本已生成......
第22条新样本已生成......
第23条新样本已生成......
第24条新样本已生成......
第25条新样本已生成......
第26条新样本已生成......
第27条新样本已生成......
第28条新样本已生成......
第29条新样本已生成......
第30条新样本已生成......
第31条新样本已生成......
第32条新样本已生成......
第33条新样本已生成......
第34条新样本已生成......
第35条新样本已生成......
第36条新样本已生成......
第37条新样本已生成......
第38条新样本已生成......
第39条新样本已生成......
第40条新样本已生成......
第41条新样本已生成......
第42条新样本已生成......
第43条新样本已生成......
第44条新样本已生成......
第45条新样本已生成......
第46条新样本已生成......
第47条新样本已生成......
第48条新样本已生成......
第49条新样本已生成......
第50条新样本已生成......
第51条新样本已生成......
第52条新样本已生成......
第53条新样本已生成......
第54条新样本已生成......
第55条新样本已生成......
第56条新样本已生成......
第57条新样本已生成......
第58条新样本已生成......
第59条新样本已生成......
第60条新样本已生成......
第61条新样本已生成......
第62条新样本已生成......
第63条新样本已生成......
第64条新样本已生成......
第65条新样本已生成......
第66条新样本已生成......
第67条新样本已生成......
第68条新样本已生成......
第69条新样本已生成......
第70条新样本已生成......
第71条新样本已生成......
第72条新样本已生成......
第73条新样本已生成....

149

In [5]:
new_sample = pd.DataFrame(data)
new_sample
# new_sample.to_excel('../qwen/all_data/excel/new_sample.xlsx', index=False)

,question,answer
0,电力系统包含哪些组成部分？,电力系统是指由发电、输电、变电、配电和用电设备构成的网络，还包括用于调节电压和频率、限制短路...
1,电网是指什么？,电力网络，通常被称为电网，是指电力系统中的一次系统，包括发电、输电、变电和配电等环节，以及相...
2,电网的作用是什么？现代电网有哪些特性？,电网作为连接电源与用户的桥梁，其核心任务是确保电力以安全、高效、经济的方式送达消费者。为了达...
3,怎样从整体角度审视电力网络？,电网的构成包括发电、输电、变电、配电和用电等设施，从电路的角度来看，这些设备可以简化为由电阻...
4,5. 电路是什么？它由哪些部分构成？,电路可以概括为电流流动的通道，它由一系列电气元件按特定顺序排列而成，主要包含电源、负载和连接...
...,...,...
144,电网调度机构的职责范围包括哪些内容？,电网调度机构的职责范围涵盖了行政管理职能、电网发电与供电的指挥、发电与用电的监管以及电力量的...
145,《电网调度管理条例》的核心指导原则是什么？该条例在哪些领域内有效应用？,《电网调度管理条例》确立了以下核心准则：\n(1) 实行统一指挥与分层负责的管理方式。\n(...
146,请问什么是“分级管理”？,“统一调度”是指在调度工作中，下级单位必须遵循上级单位的指令。这一概念的核心内容可以归纳为一...
147,148.什么是所谓的“统一调度”？,“分级管理”是指依据电网层次结构，为了界定各调度机构的职责和权力，并确保统一调度的有效执行，...


In [6]:
new_sample.to_excel('../qwen/all_data/excel/new_sample_3.xlsx', index=False)

#### 测试

In [34]:
template_word = """请你根据以下原文进行改写，要求:如果字数小于1000，直接把给你的原文输出,其余的什么都不要说，如果字数大于1000，则尽量将字数缩减至1000以内，但要保持原文的基本内容不能改变。
原文：{content}
"""
prompt_word = PromptTemplate(
    template=template_word,
    input_variables = ["content"]
)
print(prompt_word.format(content=combined_data['answer'][0]))

请你根据以下原文进行改写，要求:如果字数小于1000，直接把给你的原文输出,其余的什么都不要说，如果字数大于1000，则尽量将字数缩减至1000以内，但要保持原文的基本内容不能改变。
原文：  光伏发电是指利用太阳能光伏电池把太阳辐射能直接转变成电能的发电方式。光伏发电是当今太阳能发电的主流。
  分布式光伏系统是指在用户场地附近建设，一般接入低于 35kV 及以下电压等级的电网，所发电以就地消纳为主，且在配电系统平衡调节为特性的光伏发电设施。分布式光伏系统的运行方式自发自用、余电上网，全额上网两种方式。自发自用、余电上网是指分布式光伏发电系统所发电力由电力用户优先使用，多余电量接入电网；全额上网是指分布式光伏发电系统所发电力全部接入电网。
  分布式发电遵循因地制宜、清洁高效、分散布局、就近利用的原则，充分利用当地的太阳能资源，替代和减少化石能源消费。



In [26]:
prompt = prompt_word.format(content=combined_data['answer'][0])
baichuan_llm(prompt)

'原文字数为：269 字，小于1000字，因此直接输出原文：\n\n光伏发电是指利用太阳能光伏电池把太阳辐射能直接转变成电能的发电方式。光伏发电是当今太阳能发电的主流。\n分布式光伏系统是指在用户场地附近建设，一般接入低于 35kV 及以下电压等级的电网，所发电以就地消纳为主，且在配电系统平衡调节为特性的光伏发电设施。分布式光伏系统的运行方式自发自用、余电上网，全额上网两种方式。自发自用、余电上网是指分布式光伏发电系统所发电力由电力用户优先使用，多余电量接入电网；全额上网是指分布式光伏发电系统所发电力全部接入电网。\n分布式发电遵循因地制宜、清洁高效、分散布局、就近利用的原则，充分利用当地的太阳能资源，替代和减少化石能源消费。'

In [38]:
df = pd.read_excel("../qwen/all_data/excel/电力现货市场百问百答.xlsx")
df

,question,answer
0,1.什么是电力市场？,电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现电力商品交换...
1,2.电力市场与普通商品市场有哪些差异？,电力市场相较于普通商品市场具有显著的特殊性，归根到底就在于电力商品与其他一般商品的差异性...
2,3.电力市场有哪些特征？,电力工业的技术特征和电力商品的特殊性，导致了电力市场的固有特征。\n（1）电力市场服务的...
3,4.什么是电力市场体系？,一般而言，电力市场体系是由政府监管下各类电力细分市场有机组合而成的群体系统，包括为构建各...
4,5.电力现货市场在电力市场体系中的地位和作用是什么？,电力市场体系建设是一项系统工程，世界上绝大多数电力市场都是以电力现货市场建设起步。电力现...
...,...,...
119,121.现货市场建设为何需要完备的电能量计量系统？,现货市场建设需要完备的电能量计量系统的原因。现货市场结算需要通过采集市场计量装置的计量数...
120,122.现货市场建设对电能量计量系统有什么新的技术要求？,现货市场建设对电能量计量系统的技术要求：电力现货市场对数据采集频率、计量数据准确度、计量...
121,123.现货市场运营为什么需要对用户表计改造？,现货市场运行需要对用户表计改造的原因。现货市场开启之后，计量周期由月缩短至15min，缩...
122,124.现货市场运营对表计改造有什么要求？,对表计改造的要求。针对符合准入条件（一定电压等级或一定规模）且有参与市场意愿的用户，初期...


In [39]:
for index, (q,a) in enumerate(zip(df['question'],df['answer'])):
    prompt = prompt_word.format(content=a)
    df.loc[index,'answer'] = baichuan_llm(prompt)
    print(f"第{index+1}条已更新.......")
df

第1条已更新.......
第2条已更新.......
第3条已更新.......
第4条已更新.......
第5条已更新.......
第6条已更新.......
第7条已更新.......
第8条已更新.......
第9条已更新.......
第10条已更新.......
第11条已更新.......
第12条已更新.......
第13条已更新.......
第14条已更新.......
第15条已更新.......
第16条已更新.......
第17条已更新.......
第18条已更新.......
第19条已更新.......
第20条已更新.......
第21条已更新.......
第22条已更新.......
第23条已更新.......
第24条已更新.......
第25条已更新.......
第26条已更新.......
第27条已更新.......
第28条已更新.......
第29条已更新.......
第30条已更新.......
第31条已更新.......
第32条已更新.......
第33条已更新.......
第34条已更新.......
第35条已更新.......
第36条已更新.......
第37条已更新.......
第38条已更新.......
第39条已更新.......
第40条已更新.......
第41条已更新.......
第42条已更新.......
第43条已更新.......
第44条已更新.......
第45条已更新.......
第46条已更新.......
第47条已更新.......
第48条已更新.......
第49条已更新.......
第50条已更新.......
第51条已更新.......
第52条已更新.......
第53条已更新.......
第54条已更新.......
第55条已更新.......
第56条已更新.......
第57条已更新.......
第58条已更新.......
第59条已更新.......
第60条已更新.......
第61条已更新.......
第62条已更新.......
第63条已更新.......
第64条已更新.......
第65条已更新.......
第66条已更新.......
第67条已更新.......
第68条

,question,answer
0,1.什么是电力市场？,电力市场的定义是基于市场经济原则，旨在实现电力商品交换的电力工业组织结构、经营管理和运行规则...
1,2.电力市场与普通商品市场有哪些差异？,电力市场的特殊性主要源于电力商品与普通商品的本质区别。电力商品即电能，其特殊性体现在以下几个...
2,3.电力市场有哪些特征？,电力市场的特点源于电力行业的技术特性和电力商品的独特性质。这些特点包括：\n\n1. 电力服...
3,4.什么是电力市场体系？,原文：一般而言，电力市场体系是在政府监管下，由各类电力细分市场组成的群体系统，这些细分市场通...
4,5.电力现货市场在电力市场体系中的地位和作用是什么？,电力市场体系建设是一个复杂的系统工程，大多数电力市场都是从现货市场开始建立的。电力现货市场在...
...,...,...
119,121.现货市场建设为何需要完备的电能量计量系统？,现货市场建设需要完备的电能量计量系统，因为这关系到市场成员的利益和市场的稳定运营。在现货市场...
120,122.现货市场建设对电能量计量系统有什么新的技术要求？,原文长度为1101字，需要缩减至1000字以内。\n\n现货市场建设对电能量计量系统的技术要...
121,123.现货市场运营为什么需要对用户表计改造？,现货市场运行需要对用户表计改造的原因。现货市场开启后，计量周期从每月缩短至每15分钟，即缩短...
122,124.现货市场运营对表计改造有什么要求？,原文：对表计改造的要求。针对符合准入条件（一定电压等级或一定规模）且有参与市场意愿的用户，初...


In [40]:
df.to_excel('../qwen/all_data/excel/电力现货市场百问百答.xlsx', index=False)